In [5]:
# Standard library imports
import argparse
import csv
import copy
import json
import logging as log
from collections import Counter, deque
import itertools
from math import degrees
import os
from pathlib import Path

# Related third-party imports
import cv2
import cv2 as cv
import numpy as np
import torch

from IPython import display
import mediapipe as mp
from mediapipe import solutions
from ultralytics import YOLO

# Local application/library-specific imports
import supervision as sv
from model import PalmDetection
from utils import CvFpsCalc
from utils.utils import rotate_and_crop_rectangle

# Configuring logging
log.basicConfig(level=log.INFO)
display.clear_output()

# Define default values
desired_width = 3840
desired_height = 2160
# desired_width = 1920
# desired_height = 1080
cap_width = 640
cap_height = 480
min_detection_confidence = 0.5
min_detection_confidence_m = 0.1
disable_image_flip = False
iou_threshold = 0.1
cvFpsCalc = CvFpsCalc(buffer_len=10)
palm_trackid_cxcy = {}

# Object Detection and Counting Script Varibles 
object_counter = {}
object_counter1 = {}
northcount = 0
southcount = 0
returncount = 0
purchasecount = 0
purchasecounter =1
returncounter = 1  
count=1
count1=1 
Missing_Purchase_Item = 0
Missing_Return_Item = 0
last_obj_name = 0
last_obj_name_1 = 0
hand_bbox = []
x_max, y_max = 0, 0
prev_centroid_y = None
prev_centroid_ym = None
prev_centroid_yh = None
max_probability_value = None
direction = "No movement"
media_direction = "No movement"
hand_direction = "No movement"
object_direction = "No movement"


# Additional variables
device = 0
lines_hand = [
        [0, 1], [1, 2], [2, 3], [3, 4],
        [0, 5], [5, 6], [6, 7], [7, 8],
        [5, 9], [9, 10], [10, 11], [11, 12],
        [9, 13], [13, 14], [14, 15], [15, 16],
        [13, 17], [17, 18], [18, 19], [19, 20], [0, 17],
    ]
mode = 0
wh_ratio = cap_width / cap_height
auto = False
prev_number = -1
image = None


# Models Path 

# detection_model
model = YOLO('/home/tricubics/Desktop/Test_Script/model/classification_model/weights/best.pt')
#label_map = model.model.names


#mediapiple_hands_object.
detector = solutions.hands.Hands(min_detection_confidence= min_detection_confidence_m, min_tracking_confidence=0.1, static_image_mode=False, max_num_hands=4)

#classification_model
model1 = YOLO('/home/tricubics/Desktop/Test_Script/model/object_detection_model/weights/best.pt')
model1_names = model1.names

# Test Input 
#video_directory 
#VID_PATH = "/home/tricubics/Desktop/Test_Script/Input/one_camera_data/output_video.mp4"
VID_PATH = "/home/tricubics/Desktop/Test_Script/Input/two_camera_data/output_video2.mp4"
#VID_PATH = '/home/tricubics/Desktop/Test_Script/Input/two_camera_data/output_video3.mp4'
#VID_PATH = '/home/tricubics/Desktop/Test_Script/Input/one_camera_data/output_video.mp4'
#VID_PATH = '/home/tricubics/Desktop/Test_Script/Input/one_camera_data/output_video_2.mp4'
#VID_PATH = '/home/tricubics/Desktop/Test_Script/Input/one_camera_data/output_video_15.mp4'
#VID_PATH ='/home/tricubics/Desktop/Test_Script/Input/small_refregeator_data/1_new_fridge_right_30fps.mp4'

#VID_PATH ='/home/tricubics/Desktop/Test_Script/Input/testvideo.mp4'

#video_info
video_info = sv.VideoInfo.from_video_path(VID_PATH)

frame_resolution_wh = (1920, 1080)

#frame_directory 
#frames_dir = '/home/tricubics/Desktop/Test_Script/Input/frames/video0'
#frames_dir = '/home/tricubics/133045_299/video0'
#r_frames_dir = '/home/tricubics/Desktop/Test_Script/Input/24041102205605847583/video0'
l_frames_dir = '/home/tricubics/Desktop/Test_Script/Input/24041102205605847583/video2'

#output_frame_store_location 
#output_folder = "/home/tricubics/Desktop/Test_Script/output1"
output_folder = "/home/tricubics/Desktop/Test_Script/output/right"
os.makedirs(output_folder, exist_ok=True)

# Define and Load a Zone
def load_zones(json_path, zone_str):
    with open(json_path) as f:
        zones_dict = json.load(f)
    return np.array(zones_dict[zone_str]["points"], np.int32)

# polygon = load_zones("zones.json", "test-example-1")
# polygon1 = load_zones("zones.json", "line-zone")
# polygon2 = load_zones("zones.json", "line-zone_1")
# polygon3 = load_zones("zones.json", "line-zone_2")
# polygon7 = load_zones("zones.json", "line-zone_6")
# zone = sv.PolygonZone(polygon=polygon, frame_resolution_wh=video_info.resolution_wh)
# zone1 = sv.PolygonZone(polygon=polygon1, frame_resolution_wh=video_info.resolution_wh)
# zone2 = sv.PolygonZone(polygon=polygon2, frame_resolution_wh=video_info.resolution_wh)
# zone3 = sv.PolygonZone(polygon=polygon3, frame_resolution_wh=video_info.resolution_wh)
# # # zone4 = sv.PolygonZone(polygon=polygon4, frame_resolution_wh=video_info.resolution_wh)
# # # zone5 = sv.PolygonZone(polygon=polygon5, frame_resolution_wh=video_info.resolution_wh)
# # # zone6 = sv.PolygonZone(polygon=polygon6, frame_resolution_wh=video_info.resolution_wh)
#zone7 = sv.PolygonZone(polygon=polygon7, frame_resolution_wh=video_info.resolution_wh)

# polygon = load_zones("zones_op.json", "test-example-1")
# polygon1 = load_zones("zones_op.json", "line-zone")
# polygon2 = load_zones("zones_op.json", "line-zone_1")
# # polygon4 = load_zones("zones.json", "line-zone_3")
# # polygon5 = load_zones("zones.json", "line-zone_4")
# # polygon6 = load_zones("zones.json", "line-zone_5")
# polygon3 = load_zones("zones_op.json", "line-zone_3")

# polygon = load_zones("zones_r.json", "test-example-1")
# polygon1 = load_zones("zones_r.json", "line-zone")
# polygon2 = load_zones("zones_r.json", "line-zone_1")
# polygon3 = load_zones("zones_r.json", "line-zone_3")
# polygon4 = load_zones("zones_r.json", "line-zone_4")
# polygon5 = load_zones("zones_r.json", "line-zone_5")

polygon = load_zones("zones_l.json", "test-example-1")
polygon1 = load_zones("zones_l.json", "line-zone")
polygon2 = load_zones("zones_l.json", "line-zone_1")
polygon3 = load_zones("zones_l.json", "line-zone_3")
polygon4 = load_zones("zones_l.json", "line-zone_4")
polygon5 = load_zones("zones_l.json", "line-zone_5")

zone = sv.PolygonZone(polygon=polygon, frame_resolution_wh=frame_resolution_wh)
zone1 = sv.PolygonZone(polygon=polygon1, frame_resolution_wh=frame_resolution_wh)
zone2 = sv.PolygonZone(polygon=polygon2, frame_resolution_wh=frame_resolution_wh)
zone3 = sv.PolygonZone(polygon=polygon3, frame_resolution_wh=frame_resolution_wh)
zone4 = sv.PolygonZone(polygon=polygon4, frame_resolution_wh=video_info.resolution_wh)
zone5 = sv.PolygonZone(polygon=polygon5, frame_resolution_wh=video_info.resolution_wh)

# # zone5 = sv.PolygonZone(polygon=polygon5, frame_resolution_wh=video_info.resolution_wh)
# # zone4 = sv.PolygonZone(polygon=polygon4, frame_resolution_wh=video_info.resolution_wh)
# # zone5 = sv.PolygonZone(polygon=polygon5, frame_resolution_wh=video_info.resolution_wh)
# # zone6 = sv.PolygonZone(polygon=polygon6, frame_resolution_wh=video_info.resolution_wh)
#zone7 = sv.PolygonZone(polygon=polygon7, frame_resolution_wh=frame_resolution_wh)

zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color.white(), thickness=6, text_thickness=6, text_scale=4)
zone_annotator1 = sv.PolygonZoneAnnotator(zone=zone1, color=sv.Color.white(), thickness=6, text_thickness=6, text_scale=4)
zone_annotator2 = sv.PolygonZoneAnnotator(zone=zone2, color=sv.Color.white(), thickness=6, text_thickness=6, text_scale=4)
zone_annotator3 = sv.PolygonZoneAnnotator(zone=zone3, color=sv.Color.white(), thickness=6, text_thickness=6, text_scale=4)
# zone_annotator4 = sv.PolygonZoneAnnotator(zone=zone4, color=sv.Color.white(), thickness=6, text_thickness=6, text_scale=4)
# zone_annotator5 = sv.PolygonZoneAnnotator(zone=zone5, color=sv.Color.white(), thickness=6, text_thickness=6, text_scale=4)
# zone_annotator6 = sv.PolygonZoneAnnotator(zone=zone6, color=sv.Color.white(), thickness=6, text_thickness=6, text_scale=4)
#zone_annotator7 = sv.PolygonZoneAnnotator(zone=zone7, color=sv.Color.white(), thickness=6, text_thickness=6, text_scale=4)
zone_annotator4 = sv.PolygonZoneAnnotator(zone=zone4, color=sv.Color.white(), thickness=6, text_thickness=6, text_scale=4)
zone_annotator5 = sv.PolygonZoneAnnotator(zone=zone5, color=sv.Color.white(), thickness=6, text_thickness=6, text_scale=4)

box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)

# Final Code 
indexs=0

I0000 00:00:1712909874.211055  556250 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1712909874.306060  572762 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.54.14), renderer: NVIDIA GeForce RTX 3060/PCIe/SSE2


##Left_Frame: Return_Logic

In [6]:
direction = "No movement"
hand_direction = "No movement"
object_direction = "No movement"

with sv.VideoSink("new_det_tracking_result.mp4", video_info) as sink:
    #L1 => source_file_reading 
    jpg_files = [f for f in os.listdir(l_frames_dir) if f.endswith('.jpg')]
    jpg_files.sort()
    for jpg_file in jpg_files:
        frame = cv2.imread(os.path.join(l_frames_dir, jpg_file))
        frame = cv2.resize(frame, (desired_width, desired_height)) #VID_PATH
        #L2 => object_detection  
        #for index, result in enumerate(model.track(source = frame, show=False, stream=True, verbose=True, device=1, augment=True, persist=True)):
        # Use GPU 0 and GPU 1
        results = model.predict(frame, stream=True, device=['cuda:0'])
        # Access the first detection result using next()
        result = next(results)
        fps = cvFpsCalc.get() 
        frame = result.orig_img
        frame2 = frame
        detections = sv.Detections.from_ultralytics(result)
        bbox = result.boxes
        height, width, _ = frame.shape
        image = cv.resize(frame, (cap_width, cap_height))
        frame = image
    
        #L3 => hand_detection_B : neural_network 
        if image is not None:
            hands = palm_detection(image)
            if len(hands) == 0:
                palm_trackid_cxcy = {}

            palm_trackid_box_x1y1s = {}
            not_rotate_rects = []
            rects = []
            rects_tuple = None
            cropted_rotated_hands_images = []

            if len(hands) > 0:
                for hand in hands:
                    sqn_rr_size = hand[0]
                    rotation = hand[1]
                    sqn_rr_center_x = hand[2]
                    sqn_rr_center_y = hand[3]
                    cx = int(sqn_rr_center_x * desired_width)
                    cy = int(sqn_rr_center_y * desired_height)
                        
                    # Calculate the new width and height of the rectangle
                    new_width = sqn_rr_size * desired_width * 0.5
                    new_height = sqn_rr_size * desired_height * wh_ratio * 0.5

                    # Calculate the new bounding box coordinates
                    xmin = int(cx - new_width / 2)
                    xmax = int(cx + new_width / 2)
                    ymin = int(cy - new_height / 2)
                    ymax = int(cy + new_height / 2)

                    xmin = max(0, xmin)
                    xmax = min(desired_width, xmax)
                    ymin = max(0, ymin)
                    ymax = min(desired_height, ymax)

                    current_centroid_y = int((ymin + ymax) // 2)
                    if prev_centroid_yh is not None:
                        if current_centroid_y > prev_centroid_yh:
                            hand_direction = "South"
                        elif current_centroid_y < prev_centroid_yh:
                            hand_direction = "North"
                        else:
                            hand_direction = "No movement"
                    prev_centroid_yh = current_centroid_y  
                    degree = degrees(rotation)
                    rects.append([cx, cy, (xmax-xmin), (ymax-ymin), degree])

                rects = np.asarray(rects, dtype=np.float32)
                cropted_rotated_hands_images = rotate_and_crop_rectangle(
                    image=image,
                    rects_tmp=rects,
                    operation_when_cropping_out_of_range='padding',
                )
            
                for rect in rects:
                    rects_tuple = ((rect[0], rect[1]), (rect[2], rect[3]), rect[4])
                    box = cv.boxPoints(rects_tuple).astype(np.int0)
                    cv.drawContours(frame2, [box], 0,(0,0,255), 2, cv.LINE_AA)
            
        frame = frame2 
        #L4 => direction 
        for bbox in result.boxes:
            x_min, y_min, x_max, y_max = bbox.xyxy[0]
            current_centroid_y = (y_min + y_max) // 2
            if prev_centroid_y is not None:
                if current_centroid_y > prev_centroid_y:
                    object_direction = "South"
                elif current_centroid_y < prev_centroid_y:
                    object_direction = "North"
           
                else:
                    object_direction = "No movement"
    
            prev_centroid_y = current_centroid_y
      
        if hand_direction != "No movement" and object_direction != "No movement":
            if hand_direction == object_direction:
                direction = object_direction
            else:
                direction = object_direction  
        else:
            if object_direction != "No movement":
                direction = object_direction
            elif hand_direction != "No movement":
                direction = hand_direction    
            else:
                direction = "No movement"

        print("Final_Direction:", direction)

        # if "North" in direction:
        #     southcount = 0
        #     print("North_First_Operation")
        #     if northcount==0:
        #         purchasecount+=1
        #         print("North_Second_Operation")
        #         northcount=1
        #         #print("purchasecount : ",purchasecount)

        # elif "South" in direction:
        #     northcount = 0
        #     print("South_First_Operation")
        #     if southcount==0:
        #         print("South_Second_Operation")
        #         returncount+=1
        #         southcount=1
        #         #print("returncount :",returncount)
    
        #L5 => zone_based_frame_annotation_and_object_classification 

        #annotate_frames     
        frame = zone_annotator.annotate(scene=frame)    
        #frame = zone_annotator1.annotate(scene=frame) 
        #frame = zone_annotator2.annotate(scene=frame)  
        #frame = zone_annotator7.annotate(scene=frame) 
        frame = zone_annotator3.annotate(scene=frame) 
        frame = zone_annotator4.annotate(scene=frame) 
        #frame = zone_annotator5.annotate(scene=frame) 

        #trigger_the_zone_and_filter_detections_based_on_the_intersection_result
        mask = zone.trigger(detections=detections)
        #mask1 = zone1.trigger(detections=detections)
        #mask2 = zone2.trigger(detections=detections)
        #mask7 = zone7.trigger(detections=detections)
        mask3 = zone3.trigger(detections=detections)
        mask4 = zone4.trigger(detections=detections)
        #mask5 = zone5.trigger(detections=detections)

        
        detections_filtered = detections[mask3]
 

        if mask.any(): 
            detections_filtered = detections[mask]
        elif mask3.any(): 
            detections_filtered = detections[mask3]
        elif mask4.any():  
            detections_filtered = detections[mask4]
    

        if len(detections_filtered) > 0:
            #classification 
            box = detections_filtered.xyxy[0]  
            x1, y1, x2, y2 = int(box[0]), int(box[1]), int(box[2]), int(box[3])
            object_copy = frame[y1:y2, x1:x2]

            try:
                classification_results = model1.predict(object_copy, imgsz=224, conf=0.5, device=1, half=True)[0]
                probabilities = classification_results.probs.top1
                max_probability_value = model1_names[probabilities]
                print("max_probability_value", max_probability_value)
            except:
                print("Return to CS Window")
                pass

            first_detection = detections_filtered[0]
            labels = max_probability_value
            frame = box_annotator.annotate(scene=frame, detections=detections_filtered, labels=labels)
            obj_name = max_probability_value

            print("obj_name", obj_name)

            #L6  => Inference_Generation and result printing  
                
            if "North" in direction:
                print("Object reaches North Direction")
                last_obj_name_1 = 0
                count = 1
                # if count1 == 1:
                #     count1 = 0
                # else:
                #         if mask.any() or mask3.any() or mask4.any(): 
                #             if obj_name != last_obj_name:
                #                 purchasecounter+=1
                #                 last_obj_name = obj_name
                #                 if obj_name not in object_counter:
                #                     object_counter[obj_name] = 1
                #                     for obj, count in object_counter.items():
                #                         print(f"Object: {obj}, Counter: {count}")
                #                 else:
                #                     object_counter[obj_name] += 1
                #                     for obj, count in object_counter.items():
                #                         print(f"Object: {obj}, Counter: {count}")

            elif "South" in direction:
                last_obj_name = 0
                #count1 = 1 
                if count == 1:
                   count = 0
                else:
                    if mask.any() or mask3.any() or mask4.any():
                        #if obj_name in object_counter:
                        if obj_name != last_obj_name_1:
                            returncounter+=1 
                            last_obj_name_1 = obj_name
                            if obj_name not in object_counter1:
                                object_counter1[obj_name] = 1
                                for obj, count in object_counter1.items():
                                    print(f"Object: {obj}, Counter: {count}")
                            else:
                                object_counter1[obj_name] += 1
                                for obj, count in object_counter1.items():
                                    print(f"Object: {obj}, Counter: {count}")
                                                                                  
        # #Display Purchase counts in the top right corner
        # for idx, (key, value) in enumerate(object_counter.items()):
        #     cnt_str1 = f"{key}: {value}"
        #     print('Numbers of Purchase Item', cnt_str1)
        #     cv2.line(frame, (width - 450, 25), (width, 25), [85, 45, 255], 40)
        #     cv2.putText(frame, 'Numbers of Purchase Item', (width - 450, 35), 0, 1, [225, 255, 255], thickness=2, lineType=cv2.LINE_AA)
        #     cv2.line(frame, (width - 450, 65 + (idx * 40)), (width, 65 + (idx * 40)), [85, 45, 255], 30)
        #     cv2.putText(frame, cnt_str1, (width - 450, 75 + (idx * 40)), 0, 1, [255, 255, 255], thickness=2, lineType=cv2.LINE_AA)
        
        #Display Returns counts in the top left corner
        for idx, (key, value) in enumerate(object_counter1.items()):
            #returned_count = object_counter.get(key, 0)
            #updated_count =  returned_count - value 
            #receipt_count = updated_count if updated_count >= 0 else 0
            cnt_str = f"{key}: {value}" 
            print('Number of Returns Item', cnt_str)
            cv2.line(frame, (20, 25), (400, 25), [85, 45, 255], 40)
            cv2.putText(frame, 'Number of Returns Item', (11, 35), 0, 1, [225, 255, 255], thickness=2, lineType=cv2.LINE_AA)    
            cv2.line(frame, (20, 65 + (idx * 40)), (300, 65 + (idx * 40)), [85, 45, 255], 30)
            cv2.putText(frame, cnt_str, (11, 75 + (idx * 40)), 0, 1, [225, 255, 255], thickness=2, lineType=cv2.LINE_AA)    

        # # Display receipt
        # for idx, (key, value) in enumerate(object_counter.items()):
        #     returned_count = object_counter1.get(key, 0)
        #     updated_count = value - returned_count 
        #     receipt_count = updated_count if updated_count >= 0 else 0
        #     cnt_str1_updated = f"{key}: {receipt_count}"
        #     print('Receipt', cnt_str1_updated)
        #     cv2.line(frame, (width - 400, 525), (width, 525), [85, 45, 255], 40)
        #     cv2.putText(frame, 'Receipt', (width - 400, 535), 0, 1, [225, 255, 255], thickness=2, lineType=cv2.LINE_AA)
        #     cv2.line(frame, (width - 400, 565 + (idx * 40)), (width, 565 + (idx * 40)), [85, 45, 255], 30)
        #     cv2.putText(frame, cnt_str1_updated, (width - 400, 575 + (idx * 40)), 0, 1, [225, 255, 255], thickness=2, lineType=cv2.LINE_AA) 
        
        # if purchasecount > purchasecounter:
        #    Missing_Purchase_Item =  purchasecount - purchasecounter
        #    purchasecount=purchasecounter
        # if returncount > returncounter: 
        #    Missing_Return_Item = returncount -returncounter

        # cv2.line(frame, (width - 400, 1025), (width, 1025), [85, 45, 255], 40)
        # cv2.putText(frame, 'Missing_Purchase_Item', (width - 400, 1035), 0, 1, [225, 255, 255], thickness=2, lineType=cv2.LINE_AA)
        # cv2.line(frame, (width - 400, 1035 + (1 * 40)), (width, 1035 + (1 * 40)), [85, 45, 255], 30)
        # cv2.putText(frame,  str(Missing_Purchase_Item), (width - 400, 1045 + (1 * 40)), 0, 1, [225, 255, 255], thickness=2, lineType=cv2.LINE_AA) 
        # cv2.line(frame, (width - 400, 1225), (width, 1225), [85, 45, 255], 40)
        # cv2.putText(frame, 'Missing_Return_Item', (width - 400, 1235), 0, 1, [225, 255, 255], thickness=2, lineType=cv2.LINE_AA)
        # cv2.line(frame, (width - 400, 1235 + (1 * 40)), (width, 1235 + (1 * 40)), [85, 45, 255], 30)
        # cv2.putText(frame,  str(Missing_Return_Item), (width - 400, 1245 + (1 * 40)), 0, 1, [225, 255, 255], thickness=2, lineType=cv2.LINE_AA)   
    
        screen_width = 800
        screen_height = 600    
        resized_image = cv2.resize(frame, (screen_width, screen_height))    
        cv.imshow('Automatic_SelfCheckout_Detection', resized_image)
        key = cv.waitKey(1)
        if key == 27:  # ESC
                break
        cv2.imwrite(os.path.join(output_folder, f"frame_{indexs}.jpg"), frame) 
        indexs+=1
        #sink.write_frame(frame)
            


0: 384x640 (no detections), 11.2ms
Final_Direction: No movement

0: 384x640 (no detections), 11.7ms
Final_Direction: North

0: 384x640 (no detections), 11.1ms
Final_Direction: North

0: 384x640 (no detections), 11.2ms
Final_Direction: North

0: 384x640 (no detections), 11.0ms
Final_Direction: North

0: 384x640 2 bottles, 11.6ms
Final_Direction: North

0: 224x224 saero_soju 0.83, terra_bottle 0.15, cass_can 0.02, terra_can 0.00, cass_bottle 0.00, 1.8ms
Speed: 12.2ms preprocess, 1.8ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)
max_probability_value saero_soju
obj_name saero_soju
Object reaches North Direction

0: 384x640 2 bottles, 11.3ms
Final_Direction: North

0: 224x224 saero_soju 0.98, terra_bottle 0.02, cass_can 0.01, terra_can 0.00, cass_bottle 0.00, 2.1ms
Speed: 40.7ms preprocess, 2.1ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)
max_probability_value saero_soju
obj_name saero_soju
Object reaches North Direction

0: 384x640 2 bottles, 10

: 

##Right_Frame 

In [ ]:
direction = "No movement"
hand_direction = "No movement"
object_direction = "No movement"

with sv.VideoSink("new_det_tracking_result.mp4", video_info) as sink:
    #L1 => source_file_reading 
    jpg_files = [f for f in os.listdir(l_frames_dir) if f.endswith('.jpg')]
    jpg_files.sort()
    for jpg_file in jpg_files:
        frame = cv2.imread(os.path.join(l_frames_dir, jpg_file))
        frame = cv2.resize(frame, (desired_width, desired_height)) #VID_PATH
        #L2 => object_detection  
        #for index, result in enumerate(model.track(source = frame, show=False, stream=True, verbose=True, device=1, augment=True, persist=True)):
        # Use GPU 0 and GPU 1
        results = model.predict(frame, stream=True, device=['cuda:0'])
        # Access the first detection result using next()
        result = next(results)
        fps = cvFpsCalc.get() 
        frame = result.orig_img
        frame2 = frame
        detections = sv.Detections.from_ultralytics(result)
        bbox = result.boxes
        height, width, _ = frame.shape
        image = cv.resize(frame, (cap_width, cap_height))
        frame = image
    
        #L3 => hand_detection_B : neural_network 
        if image is not None:
            hands = palm_detection(image)
            if len(hands) == 0:
                palm_trackid_cxcy = {}

            palm_trackid_box_x1y1s = {}
            not_rotate_rects = []
            rects = []
            rects_tuple = None
            cropted_rotated_hands_images = []

            if len(hands) > 0:
                for hand in hands:
                    sqn_rr_size = hand[0]
                    rotation = hand[1]
                    sqn_rr_center_x = hand[2]
                    sqn_rr_center_y = hand[3]
                    cx = int(sqn_rr_center_x * desired_width)
                    cy = int(sqn_rr_center_y * desired_height)
                        
                    # Calculate the new width and height of the rectangle
                    new_width = sqn_rr_size * desired_width * 0.5
                    new_height = sqn_rr_size * desired_height * wh_ratio * 0.5

                    # Calculate the new bounding box coordinates
                    xmin = int(cx - new_width / 2)
                    xmax = int(cx + new_width / 2)
                    ymin = int(cy - new_height / 2)
                    ymax = int(cy + new_height / 2)

                    xmin = max(0, xmin)
                    xmax = min(desired_width, xmax)
                    ymin = max(0, ymin)
                    ymax = min(desired_height, ymax)

                    current_centroid_y = int((ymin + ymax) // 2)
                    if prev_centroid_yh is not None:
                        if current_centroid_y > prev_centroid_yh:
                            hand_direction = "South"
                        elif current_centroid_y < prev_centroid_yh:
                            hand_direction = "North"
                        else:
                            hand_direction = "No movement"
                    prev_centroid_yh = current_centroid_y  
                    degree = degrees(rotation)
                    rects.append([cx, cy, (xmax-xmin), (ymax-ymin), degree])

                rects = np.asarray(rects, dtype=np.float32)
                cropted_rotated_hands_images = rotate_and_crop_rectangle(
                    image=image,
                    rects_tmp=rects,
                    operation_when_cropping_out_of_range='padding',
                )
            
                for rect in rects:
                    rects_tuple = ((rect[0], rect[1]), (rect[2], rect[3]), rect[4])
                    box = cv.boxPoints(rects_tuple).astype(np.int0)
                    cv.drawContours(frame2, [box], 0,(0,0,255), 2, cv.LINE_AA)
            
        frame = frame2 
        #L4 => direction 
        for bbox in result.boxes:
            x_min, y_min, x_max, y_max = bbox.xyxy[0]
            current_centroid_y = (y_min + y_max) // 2
            if prev_centroid_y is not None:
                if current_centroid_y > prev_centroid_y:
                    object_direction = "South"
                elif current_centroid_y < prev_centroid_y:
                    object_direction = "North"
           
                else:
                    object_direction = "No movement"
    
            prev_centroid_y = current_centroid_y
      
        if hand_direction != "No movement" and object_direction != "No movement":
            if hand_direction == object_direction:
                direction = object_direction
            else:
                direction = object_direction  
        else:
            if object_direction != "No movement":
                direction = object_direction
            elif hand_direction != "No movement":
                direction = hand_direction    
            else:
                direction = "No movement"

        print("Final_Direction:", direction)

        # if "North" in direction:
        #     southcount = 0
        #     print("North_First_Operation")
        #     if northcount==0:
        #         purchasecount+=1
        #         print("North_Second_Operation")
        #         northcount=1
        #         #print("purchasecount : ",purchasecount)

        # elif "South" in direction:
        #     northcount = 0
        #     print("South_First_Operation")
        #     if southcount==0:
        #         print("South_Second_Operation")
        #         returncount+=1
        #         southcount=1
        #         #print("returncount :",returncount)
    
        #L5 => zone_based_frame_annotation_and_object_classification 

        #annotate_frames     
        frame = zone_annotator.annotate(scene=frame)    
        #frame = zone_annotator1.annotate(scene=frame) 
        #frame = zone_annotator2.annotate(scene=frame)  
        #frame = zone_annotator7.annotate(scene=frame) 
        frame = zone_annotator3.annotate(scene=frame) 
        frame = zone_annotator4.annotate(scene=frame) 
        #frame = zone_annotator5.annotate(scene=frame) 

        #trigger_the_zone_and_filter_detections_based_on_the_intersection_result
        mask = zone.trigger(detections=detections)
        #mask1 = zone1.trigger(detections=detections)
        #mask2 = zone2.trigger(detections=detections)
        #mask7 = zone7.trigger(detections=detections)
        mask3 = zone3.trigger(detections=detections)
        mask4 = zone4.trigger(detections=detections)
        #mask5 = zone5.trigger(detections=detections)

        
        detections_filtered = detections[mask3]
 

        if mask.any(): 
            detections_filtered = detections[mask]
        elif mask3.any(): 
            detections_filtered = detections[mask3]
        elif mask4.any():  
            detections_filtered = detections[mask4]
    

        if len(detections_filtered) > 0:
            #classification 
            box = detections_filtered.xyxy[0]  
            x1, y1, x2, y2 = int(box[0]), int(box[1]), int(box[2]), int(box[3])
            object_copy = frame[y1:y2, x1:x2]

            try:
                classification_results = model1.predict(object_copy, imgsz=224, conf=0.5, device=1, half=True)[0]
                probabilities = classification_results.probs.top1
                max_probability_value = model1_names[probabilities]
                print("max_probability_value", max_probability_value)
            except:
                print("Return to CS Window")
                pass

            first_detection = detections_filtered[0]
            labels = max_probability_value
            frame = box_annotator.annotate(scene=frame, detections=detections_filtered, labels=labels)
            obj_name = max_probability_value

            print("obj_name", obj_name)

            #L6  => Inference_Generation and result printing  
                
            if "North" in direction:
                print("Object reaches North Direction")
                #last_obj_name_1 = 0
                #count = 1
                if count1 == 1:
                    count1 = 0
                else:
                        if mask.any() or mask3.any() or mask4.any(): #or mask2.any() or mask3.any():
                            if obj_name != last_obj_name:
                                purchasecounter+=1
                                last_obj_name = obj_name
                                if obj_name not in object_counter:
                                    object_counter[obj_name] = 1
                                    for obj, count in object_counter.items():
                                        print(f"Object: {obj}, Counter: {count}")
                                else:
                                    object_counter[obj_name] += 1
                                    for obj, count in object_counter.items():
                                        print(f"Object: {obj}, Counter: {count}")

            elif "South" in direction:
                last_obj_name = 0
                count1 = 1 
                #if count == 1:
                #    count = 0
            #     else:
            #         if mask.any() or mask2.any() or mask3.any():
            #             if obj_name in object_counter:
            #                 if obj_name != last_obj_name_1:
            #                     returncounter+=1 
            #                     last_obj_name_1 = obj_name
            #                     if obj_name not in object_counter1:
            #                         object_counter1[obj_name] = 1
            #                         for obj, count in object_counter1.items():
            #                             print(f"Object: {obj}, Counter: {count}")
            #                     else:
            #                         object_counter1[obj_name] += 1
            #                         for obj, count in object_counter1.items():
            #                             print(f"Object: {obj}, Counter: {count}")
                                                                                  
        #Display Purchase counts in the top right corner
        for idx, (key, value) in enumerate(object_counter.items()):
            cnt_str1 = f"{key}: {value}"
            print('Numbers of Purchase Item', cnt_str1)
            cv2.line(frame, (width - 450, 25), (width, 25), [85, 45, 255], 40)
            cv2.putText(frame, 'Numbers of Purchase Item', (width - 450, 35), 0, 1, [225, 255, 255], thickness=2, lineType=cv2.LINE_AA)
            cv2.line(frame, (width - 450, 65 + (idx * 40)), (width, 65 + (idx * 40)), [85, 45, 255], 30)
            cv2.putText(frame, cnt_str1, (width - 450, 75 + (idx * 40)), 0, 1, [255, 255, 255], thickness=2, lineType=cv2.LINE_AA)
        
        # #Display Returns counts in the top left corner
        # for idx, (key, value) in enumerate(object_counter1.items()):
        #     returned_count = object_counter.get(key, 0)
        #     updated_count =  returned_count - value 
        #     receipt_count = updated_count if updated_count >= 0 else 0
        #     cnt_str = f"{key}: {value}" 
        #     print('Number of Returns Item', cnt_str)
        #     cv2.line(frame, (20, 25), (400, 25), [85, 45, 255], 40)
        #     cv2.putText(frame, 'Number of Returns Item', (11, 35), 0, 1, [225, 255, 255], thickness=2, lineType=cv2.LINE_AA)    
        #     cv2.line(frame, (20, 65 + (idx * 40)), (300, 65 + (idx * 40)), [85, 45, 255], 30)
        #     cv2.putText(frame, cnt_str, (11, 75 + (idx * 40)), 0, 1, [225, 255, 255], thickness=2, lineType=cv2.LINE_AA)    

        # # Display receipt
        # for idx, (key, value) in enumerate(object_counter.items()):
        #     returned_count = object_counter1.get(key, 0)
        #     updated_count = value - returned_count 
        #     receipt_count = updated_count if updated_count >= 0 else 0
        #     cnt_str1_updated = f"{key}: {receipt_count}"
        #     print('Receipt', cnt_str1_updated)
        #     cv2.line(frame, (width - 400, 525), (width, 525), [85, 45, 255], 40)
        #     cv2.putText(frame, 'Receipt', (width - 400, 535), 0, 1, [225, 255, 255], thickness=2, lineType=cv2.LINE_AA)
        #     cv2.line(frame, (width - 400, 565 + (idx * 40)), (width, 565 + (idx * 40)), [85, 45, 255], 30)
        #     cv2.putText(frame, cnt_str1_updated, (width - 400, 575 + (idx * 40)), 0, 1, [225, 255, 255], thickness=2, lineType=cv2.LINE_AA) 
        
        # if purchasecount > purchasecounter:
        #    Missing_Purchase_Item =  purchasecount - purchasecounter
        #    purchasecount=purchasecounter
        # if returncount > returncounter: 
        #    Missing_Return_Item = returncount -returncounter

        # cv2.line(frame, (width - 400, 1025), (width, 1025), [85, 45, 255], 40)
        # cv2.putText(frame, 'Missing_Purchase_Item', (width - 400, 1035), 0, 1, [225, 255, 255], thickness=2, lineType=cv2.LINE_AA)
        # cv2.line(frame, (width - 400, 1035 + (1 * 40)), (width, 1035 + (1 * 40)), [85, 45, 255], 30)
        # cv2.putText(frame,  str(Missing_Purchase_Item), (width - 400, 1045 + (1 * 40)), 0, 1, [225, 255, 255], thickness=2, lineType=cv2.LINE_AA) 
        # cv2.line(frame, (width - 400, 1225), (width, 1225), [85, 45, 255], 40)
        # cv2.putText(frame, 'Missing_Return_Item', (width - 400, 1235), 0, 1, [225, 255, 255], thickness=2, lineType=cv2.LINE_AA)
        # cv2.line(frame, (width - 400, 1235 + (1 * 40)), (width, 1235 + (1 * 40)), [85, 45, 255], 30)
        # cv2.putText(frame,  str(Missing_Return_Item), (width - 400, 1245 + (1 * 40)), 0, 1, [225, 255, 255], thickness=2, lineType=cv2.LINE_AA)   
    
        screen_width = 800
        screen_height = 600    
        resized_image = cv2.resize(frame, (screen_width, screen_height))    
        cv.imshow('Automatic_SelfCheckout_Detection', resized_image)
        key = cv.waitKey(1)
        if key == 27:  # ESC
                break
        cv2.imwrite(os.path.join(output_folder, f"frame_{indexs}.jpg"), frame) 
        indexs+=1
        #sink.write_frame(frame)
            

In [ ]:
#